# Exploring ICS ATT&CK

## Query ATT&CK

### Import TAXII Libraries
ATT&CK users can use the initial Server class to instantiate a server object pointing to the framework’s public TAXII server URL https://cti-taxii.mitre.org/taxii/

In [1]:
from taxii2client.v20 import Server

In [2]:
server = Server("https://cti-taxii.mitre.org/taxii/")

Available API Roots can be referenced from the server object. API Roots are logical groupings of TAXII Channels and Collections and can be thought of as instances of the TAXII API available at different URLs, where each API Root is the “root” URL of that particular instance of the TAXII API:

In [3]:
server.api_roots

In [4]:
api_root = server.api_roots[0]

### Explore ATT&CK TAXII Collections
The **collections** attribute can then be used and get more information about them via their respective available properties:

In [5]:
api_root.collections

In [6]:
for collection in api_root.collections:
    print(collection.title, "->", collection.description)

Enterprise ATT&CK -> This data collection holds STIX objects from Enterprise ATT&CK
PRE-ATT&CK -> This data collection holds STIX objects from PRE-ATT&CK
Mobile ATT&CK -> This data collection holds STIX objects from Mobile ATT&CK
ICS ATT&CK -> This data collection holds STIX objects from ICS ATT&CK


In [7]:
api_root.collections[3].title

'ICS ATT&CK'

In [8]:
api_root.collections[3].id

'02c3ef24-9cd4-48f3-a99f-b74ce24f1d34'

## Set ICS ATT&CK TAXII Collection ID Variable

In [9]:
ICS_ATTACK = "02c3ef24-9cd4-48f3-a99f-b74ce24f1d34"

## Initialize TAXII Collection Sources
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), the [TAXIICollectionSource API](https://stix2.readthedocs.io/en/latest/api/datastore/stix2.datastore.taxii.html#stix2.datastore.taxii.TAXIICollectionSource) provides an interface for searching/retrieving STIX objects from a local/remote TAXII Collection endpoint. In our case, we are pointing to our ATT&CK TAXII Collection instances (https://cti-taxii.mitre.org/stix/collections/<Collection ID>)

In [10]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v20 import Collection

In [11]:
ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
ICS_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ICS_ATTACK + "/")
TC_ICS_SOURCE = TAXIICollectionSource(ICS_COLLECTION)

## Retrieve all ICS Techniques
Now that we can query the ICS ATT&CK TAXIICollection. We can use the query method and a set of filter to retrieve STIX objects of type "attack-pattern" -> "Techniques"

In [12]:
ICS_TECHNIQUES = TC_ICS_SOURCE.query(Filter("type", "=", "attack-pattern"))
print(ICS_TECHNIQUES[0])

{
    "type": "attack-pattern",
    "id": "attack-pattern--2fedbe69-581f-447d-8a78-32ee7db939a9",
    "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5",
    "created": "2021-04-13T12:45:26.506Z",
    "modified": "2021-04-13T12:45:26.506Z",
    "name": "Remote System Information Discovery",
    "description": "An adversary may attempt to get detailed information about remote systems and their peripherals, such as make/model, role, and configuration. Adversaries may use information from Remote System Information Discovery to aid in targeting and shaping follow-on behaviors. For example, the system\u2019s operational role and model information can dictate whether it is a relevant target for the adversary\u2019s operational objectives. In addition, the system\u2019s configuration may be used to scope subsequent technique usage. Requests for system information are typically implemented using automation and management protocols and are often automatically requested by vendor

In [13]:
for TECHNIQUE in ICS_TECHNIQUES:
    print(TECHNIQUE['external_references'][0]['external_id'], "--", TECHNIQUE['name'])

T0888 -- Remote System Information Discovery
T0834 -- Native API
T0890 -- Exploitation for Privilege Escalation
T0821 -- Modify Controller Tasking
T0889 -- Modify Program
T0886 -- Remote Services
T0837 -- Loss of Protection
T0878 -- Alarm Suppression
T0806 -- Brute Force I/O
T0884 -- Connection Proxy
T0811 -- Data from Information Repositories
T0868 -- Detect Operating Mode
T0871 -- Execution through API
T0822 -- External Remote Services
T0872 -- Indicator Removal on Host
T0828 -- Loss of Productivity and Revenue
T0835 -- Manipulate I/O Image
T0801 -- Monitor Process State
T0843 -- Program Download
T0867 -- Lateral Tool Transfer
T0854 -- Serial Connection Enumeration
T0862 -- Supply Chain Compromise
T0859 -- Valid Accounts
T0802 -- Automated Collection
T0875 -- Change Program State
T0808 -- Control Device Identification
T0812 -- Default Credentials
T0870 -- Detect Program State
T0819 -- Exploit Public-Facing Application
T0823 -- Graphical User Interface
T0883 -- Internet Accessible Dev

## ICS ATT&CK Available in attackcti 0.3.4.3
Reference: https://pypi.org/project/attackcti/

In [14]:
from attackcti import attack_client
lift = attack_client()

ICS_TECHNIQUES = lift.get_ics_techniques()
print("Techniques Count:",len(ICS_TECHNIQUES))

Techniques Count: 89


In [15]:
print(ICS_TECHNIQUES[0])

{
    "type": "attack-pattern",
    "id": "attack-pattern--2fedbe69-581f-447d-8a78-32ee7db939a9",
    "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5",
    "created": "2021-04-13T12:45:26.506Z",
    "modified": "2021-04-13T12:45:26.506Z",
    "name": "Remote System Information Discovery",
    "description": "An adversary may attempt to get detailed information about remote systems and their peripherals, such as make/model, role, and configuration. Adversaries may use information from Remote System Information Discovery to aid in targeting and shaping follow-on behaviors. For example, the system\u2019s operational role and model information can dictate whether it is a relevant target for the adversary\u2019s operational objectives. In addition, the system\u2019s configuration may be used to scope subsequent technique usage. Requests for system information are typically implemented using automation and management protocols and are often automatically requested by vendor

## Get All Data Sources Mapped to ICS ATT&CK Techniques

In [16]:
ICS_DATA_SOURCES = []
for TECHNIQUE in ICS_TECHNIQUES:
    if 'x_mitre_data_sources' in TECHNIQUE.keys():
        for DS in TECHNIQUE['x_mitre_data_sources']:
            if DS not in ICS_DATA_SOURCES:
                ICS_DATA_SOURCES.append(DS)
ICS_DATA_SOURCES

['Network protocol analysis',
 'Packet capture',
 'API monitoring',
 'Process monitoring',
 'System calls',
 'Sequential event recorder',
 'Controller program',
 'Windows event logs',
 'Authentication logs',
 'Alarm history',
 'Alarm thresholds',
 'Data historian',
 'Netflow/Enclave netflow',
 'Process use of network',
 'Application logs',
 'Data loss prevention',
 'Third-party application logs',
 'File monitoring',
 'Process command-line parameters',
 'Network device logs',
 'Host network interfaces',
 'Web proxy',
 'Detonation chamber',
 'Digital signatures',
 'Web logs',
 'Web application firewall logs',
 'Binary file metadata',
 'Asset management',
 'Windows Registry',
 'Network intrusion detection system',
 'Alarm History',
 'process use of network',
 'SSl/TLS inspection',
 'File Monitoring',
 'Windows error reporting',
 'Windows registry',
 'Controller parameters',
 'Malware reverse engineering',
 'Anti-virus',
 'Email gateway',
 'Mail server']

## Get All Groups from ICS ATT&CK

In [17]:
ICS_GROUPS = lift.get_ics_groups()
for GROUP in ICS_GROUPS:
    print(GROUP['name'])

TEMP.Veles
HEXANE
Dragonfly 2.0
APT33
OilRig
Dragonfly
Sandworm Team
Lazarus Group
ALLANITE


## Get All Malware from ICS ATT&CK

In [18]:
ICS_MALWARE = lift.get_ics_malware()
for MALWARE in ICS_MALWARE:
    print(MALWARE['name'])

EKANS
Ryuk
LockerGoga
PLC-Blaster
Stuxnet
VPNFilter
NotPetya
Triton
WannaCry
Flame
Industroyer
Killdisk
Bad Rabbit
Conficker
Backdoor.Oldrea
ACAD/Medre.A
BlackEnergy
Duqu
